In [118]:
%pip install numpy
%pip install scipy
%pip install matplotlib
%pip install neurokit2 
%pip install pandas
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\arche\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [119]:
def calculate_heart_features(signal, sampling_rate):
    """Calculate heart-related features such as BPM, RR intervals, and others using Pan-Tompkins method."""
    try:
        # Detect R-peaks using NeuroKit2's Pan-Tompkins based method (implemented in ECG process)
        signals, info = nk.ecg_process(signal, sampling_rate=sampling_rate)
        rpeaks = info['ECG_R_Peaks']

        # Check if there are enough R-peaks
        if len(rpeaks) < 2:
            print(f"⚠️ Not enough R-peaks detected. Found: {len(rpeaks)}")
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ""

        # RR intervals
        try:
            rr_intervals = np.diff(rpeaks) / sampling_rate
        except ZeroDivisionError:
            print("⚠️ ZeroDivisionError while calculating RR intervals.")
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ""

        if len(rr_intervals) == 0:
            print("⚠️ No RR intervals calculated.")
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ""

        # BPM Calculation
        bpm = np.nan
        try:
            if len(rr_intervals) > 0:
                bpm = 60 / np.mean(rr_intervals)
        except ZeroDivisionError:
            print("⚠️ ZeroDivisionError while calculating BPM.")
            bpm = np.nan

        # RR Interval stats (average, min, max, median)
        try:
            average_rr = np.mean(rr_intervals)
            min_rr = np.min(rr_intervals)
            max_rr = np.max(rr_intervals)
            median_rr = np.median(rr_intervals)
        except Exception as e:
            print(f"⚠️ Error calculating RR interval stats: {e}")
            average_rr = min_rr = max_rr = median_rr = np.nan

        # PT Interval (approximate)
        pt_interval = average_rr

        # Local RR differences (sliding window of 10)
        local_rr = []
        try:
            if len(rr_intervals) > 10:
                local_rr = [abs(rr_intervals[i] - rr_intervals[i+10]) for i in range(len(rr_intervals)-10)]
        except Exception as e:
            print(f"⚠️ Error calculating local RR intervals: {e}")
            local_rr = []

        return bpm, average_rr, min_rr, max_rr, pt_interval, median_rr, ','.join(map(str, local_rr))

    except Exception as e:
        print(f"❌ Error calculating heart features: {e}")
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ""


def process_single_ecg(file_path, header_path, sampling_rate=500, downsample_factor=10, plot_signal=False, failed_log_file="failed_ecg_files.log"):
    """Process one ECG file and return extracted features."""
    try:
        mat_data = scipy.io.loadmat(file_path)
        ecg_data = mat_data['val']
        
        # Select Lead II (assuming second lead)
        lead_II = ecg_data[1]

        # Preprocess
        lead_II = preprocess_signal(lead_II, original_fs=sampling_rate, downsample_factor=downsample_factor)
        new_sampling_rate = sampling_rate // downsample_factor

        # Extract features
        bpm, avg_rr, min_rr, max_rr, pt_interval, median_rr, local_rr = calculate_heart_features(lead_II, new_sampling_rate)

        # If R-peak detection failed, log the file and return NaN values
        if np.isnan(bpm):
            with open(failed_log_file, "a") as log:
                log.write(f"Failed to process {file_path}\n")

            header_info = read_header_info(header_path)
            return {
                'PersonID': os.path.splitext(os.path.basename(file_path))[0],
                'Age': header_info['Age'],
                'Sex': header_info['Sex'],
                'Arrhythmia': header_info['Arrhythmia'],
                'BPM': np.nan,
                'Average_RR_Interval': np.nan,
                'Min_RR_Interval': np.nan,
                'Max_RR_Interval': np.nan,
                'PT_Interval': np.nan,
                'RR_Median': np.nan,
                'Local_RR_Intervals': np.nan
            }

        # If successful, proceed with the normal processing
        header_info = read_header_info(header_path)

        if plot_signal:
            plot_ecg(lead_II, new_sampling_rate, title=os.path.basename(file_path))

        return {
            'PersonID': os.path.splitext(os.path.basename(file_path))[0],
            'Age': header_info['Age'],
            'Sex': header_info['Sex'],
            'Arrhythmia': header_info['Arrhythmia'],
            'BPM': bpm,
            'Average_RR_Interval': avg_rr,
            'Min_RR_Interval': min_rr,
            'Max_RR_Interval': max_rr,
            'PT_Interval': pt_interval,
            'RR_Median': median_rr,
            'Local_RR_Intervals': local_rr
        }
    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")
        return None


In [120]:
def main():
    # Call the function to process all ECG files with multithreading
    df = process_all_ecg_files(
        base_path=".",         # path where PhysioNet is located
        downsample_factor=10,  # downsample factor
        output_csv="ecg_data_summary.csv",  # output CSV file name
        plot_signals=False,    # set True to plot each ECG
        num_threads=6         # number of threads for parallel processing (adjust as needed)
    )
    return df

In [ ]:
df = main()

✅ Found PhysioNet folder: .\physionet.org

📈 Found 45152 ECG files to process.



Processing ECG files:   0%|                                                                                                                                                  | 37/45152 [00:01<34:26, 21.83it/s]

❌ Error calculating heart features: integer division or modulo by zero


Processing ECG files:   0%|▎                                                                                                                                                 | 95/45152 [00:04<32:23, 23.19it/s]

❌ Error calculating heart features: integer division or modulo by zero


...

Processing ECG files:   0%|▋                                                                                                                                                | 214/45152 [00:09<33:29, 22.37it/s]

❌ Error calculating heart features: cannot convert float NaN to integer


Processing ECG files:   1%|▉                                                                                                                                                | 300/45152 [00:13<31:14, 23.93it/s]

❌ Error calculating heart features: integer division or modulo by zero


Processing ECG files:   1%|█                                                                                                                                                | 314/45152 [00:13<31:14, 23.92it/s]